## Method 1: Brute-force 

## Method 2: Using built-in `sorted` function.

In [10]:
class DebtSimplifier1:
    def __init__(self):
        self.debts = []

    def add_debt(self, borrower, lender, amount):
        self.debts.append((borrower, lender, amount))
        self.net_balance = {}
        for borrower, lender, amount in self.debts:
            self.net_balance[borrower] = self.net_balance.get(borrower, 0) - amount
            self.net_balance[lender] = self.net_balance.get(lender, 0) + amount
    
    def simplify_debts(self):
        net_balance_copy = self.net_balance.copy()
        sorted_balances = sorted(net_balance_copy.items(), key=lambda x: x[1]) # Here we used sorted function
    
        debtors = 0
        creditors = len(sorted_balances) - 1
        transactions = []
    
        while debtors < creditors:
            debtor, debtor_balance = sorted_balances[debtors]
            creditor, creditor_balance = sorted_balances[creditors]
    
            if debtor_balance == 0:
                debtors += 1
                continue
            if creditor_balance == 0:
                creditors -= 1
                continue
    
            transfer_amount = min(-debtor_balance, creditor_balance)
            debtor_balance += transfer_amount
            creditor_balance -= transfer_amount
    
            transactions.append((debtor, creditor, transfer_amount))
    
            sorted_balances[debtors] = (debtor, debtor_balance)
            sorted_balances[creditors] = (creditor, creditor_balance)
    
            if debtor_balance == 0:
                debtors += 1
            if creditor_balance == 0:
                creditors -= 1
    
        return transactions
    
# Example usage:
simplifier1 = DebtSimplifier1()
simplifier1.add_debt("Alice", "Bob", 40)
simplifier1.add_debt("Bob", "Charlie", 20)
simplifier1.add_debt("Charlie", "David", 50)
simplifier1.add_debt("Fred", "Bob", 10)
simplifier1.add_debt("Fred", "Charlie", 30)
simplifier1.add_debt("Fred", "David", 30)
simplifier1.add_debt("Fred", "Gabe", 10)
simplifier1.add_debt("Gabe", "Alice", 30)
simplifier1.add_debt("Gabe", "Charlie", 10)
simplifier1.add_debt("Gabe", "Fred", 20)


simplified_transactions1 = simplifier1.simplify_debts()
print("The optimal transaction is")
for debtor, creditor, amount in simplified_transactions1:
    print(f"{debtor} owes {creditor} ${amount}")


The optimal transaction is
Fred owes David $60
Gabe owes David $20
Gabe owes Bob $30
Alice owes Charlie $10


In [4]:
len(simplified_transactions1)

4

## Method 3: Using merge sort algorithm

In [11]:
class DebtSimplifier2:
    def __init__(self):
        self.debts = []

    def add_debt(self, borrower, lender, amount):
        self.debts.append((borrower, lender, amount))
        self.net_balance = {}
        for borrower, lender, amount in self.debts:
            self.net_balance[borrower] = self.net_balance.get(borrower, 0) - amount
            self.net_balance[lender] = self.net_balance.get(lender, 0) + amount
    def merge_sort(self,arr, compare_func=None):
        arr_temp = list(arr)
        n = len(arr_temp)    
    
        if n > 1: 
            mid = n // 2
            arr_temp_left = arr_temp[:mid] 
            arr_temp_right = arr_temp[mid:]
      
            arr_temp_left = self.merge_sort(arr_temp_left, compare_func)
            arr_temp_right = self.merge_sort(arr_temp_right, compare_func)
              
            i = j = k = 0
            n_left, n_right = len(arr_temp_left), len(arr_temp_right)
              
            while i < n_left and j < n_right: 
                if compare_func(arr_temp_left[i], arr_temp_right[j]):
                    arr_temp[k] = arr_temp_left[i] 
                    i += 1
                else: 
                    arr_temp[k] = arr_temp_right[j] 
                    j += 1
                k += 1
              
            while i < n_left: 
                arr_temp[k] = arr_temp_left[i] 
                i += 1
                k += 1
     
            while j < n_right: 
                arr_temp[k] = arr_temp_right[j] 
                j += 1
                k += 1
                
        return arr_temp

    def merge_sort_dict_by_value(self,dictionary):

        def compare_dict_items(item1, item2):

            return item1[1] < item2[1]
    
        items = list(dictionary.items())
        sorted_items = self.merge_sort(items, compare_func=compare_dict_items)
        return dict(sorted_items)
    
    def simplify_debts(self):
        net_balance_copy = self.net_balance.copy()
        sorted_balances = list(self.merge_sort_dict_by_value(net_balance_copy).items())
        debtors = 0
        creditors = len(sorted_balances) - 1
        transactions = []
        

        while debtors < creditors:
            debtor, debtor_balance = sorted_balances[debtors]
            creditor, creditor_balance = sorted_balances[creditors]
    
            if debtor_balance == 0:
                debtors += 1
                continue
            if creditor_balance == 0:
                creditors -= 1
                continue
    
            transfer_amount = min(-debtor_balance, creditor_balance)
            debtor_balance += transfer_amount
            creditor_balance -= transfer_amount
    
            transactions.append((debtor, creditor, transfer_amount))
    
            sorted_balances[debtors] = (debtor, debtor_balance)
            sorted_balances[creditors] = (creditor, creditor_balance)
    
            if debtor_balance == 0:
                debtors += 1
            if creditor_balance == 0:
                creditors -= 1
    
        return transactions
            
        
        
simplifier2 = DebtSimplifier2()
simplifier2.add_debt("Alice", "Bob", 40)
simplifier2.add_debt("Bob", "Charlie", 20)
simplifier2.add_debt("Charlie", "David", 50)
simplifier2.add_debt("Fred", "Bob", 10)
simplifier2.add_debt("Fred", "Charlie", 30)
simplifier2.add_debt("Fred", "David", 30)
simplifier2.add_debt("Fred", "Gabe", 10)
simplifier2.add_debt("Gabe", "Alice", 30)
simplifier2.add_debt("Gabe", "Charlie", 10)
simplifier2.add_debt("Gabe", "Fred", 20)

simplified_transactions2 = simplifier2.simplify_debts()
print("The optimal transaction is")
for debtor, creditor, amount in simplified_transactions2:
    print(f"{debtor} owes {creditor} ${amount}")
        
        
        
        

The optimal transaction is
Fred owes David $60
Gabe owes David $20
Gabe owes Bob $30
Alice owes Charlie $10


In [12]:
import timeit
method1 = timeit.timeit(lambda:simplifier1.simplify_debts() ,number=100000) * len(simplified_transactions1)
method2 = timeit.timeit(lambda: simplifier2.simplify_debts(),number=100000) * len(simplified_transactions2)
print(method1)
print(method2)

4.946660800000245
15.640314399999625
